# Basic RBM gammapy analysis for VERITAS data

Versions: gammapy-tools (1.0.0), gammapy (1.2 + maps.py patch) and V2DL3 (0.5)

This notebook is version 0.3

Uses [rbm.py](../scripts/rbm.py) and [data_products.py](../scripts/data_products.py) for 2D RBM and 1D spectral/LC analyses, respectively.

Optionally, [makeBackground.py](../background_from_bkg/makeBackground/makeBackground.py) can be used to generate bkg headers here. bkg generation and attachment can also be done in [the prepare_dataset notebook](prepare_dataset.ipynb). 

**If you already have DL3 files with bkg headers, you can skip step 2.**

## Important setup instructions

For gammapy $\leq$ 1.2, you need to replace `maps/maps.py` with [this file](https://github.com/gammapy/gammapy/blob/dbed1a1774c1f5df65d5459a0596c41c6d0bffe6/gammapy/datasets/map.py) (to be implemented in gammapy 1.3) in your gammapy install location. The `gammapy.__file__` output below should point to where gammapy is installed. 

Additionally, your `config["io"]["search_datastore"]` should have runwise bkgs already appended so that `get_background_for_run` can use KL divergence can find the nearest bkgs in parameter space. Instructions on how to do that can be found in [these docs](../docs/backgrounds.md)

Information on setting up your config file (which will tell gammapy how to run your analysis) can be found in [these docs](../docs/config.md) or by using the [example config](example_config.yaml).

In [3]:
import sys
import numpy as np
import yaml
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import os

import astropy.units as u
from astropy.io import fits

from gammapy_tools.templates import get_config
from gammapy_tools.make_background import *
import gammapy_tools.make_background as bkg
from gammapy_tools.analysis import *

from gammapy.data import DataStore

import warnings
warnings.filterwarnings("ignore")

In [2]:
import gammapy_tools
gammapy_tools.__file__

'/raid/reedbuck/lrizk/envs/gammapy/lib/python3.11/site-packages/gammapy_tools/__init__.py'

## 1. Set up bkg and analysis configuration

***Note: bkg $\equiv$ radial acceptance!***

This should be a "full" config file containing background generation information as well as analysis (RBM + 1D spectra and lightcurve) information. 

An example config file is [example_config.yaml](example_config.yaml). Instructions for setting up the config file are listed as comments within the example file. 

Even if you have generated bkg headers elsewhere, the `'io'`, `'exclusion'`, `'analysis_selection'`, and `'run_selection'` parameters are re-used in the scripts needed to run this notebook.

In [4]:
with open('example_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [5]:
config

{'io': {'runlist': '/raid/romulus/swong/ED490/Eventdisplay_AnalysisScripts_VTS/scripts/crab_test.txt',
  'search_datastore': '/raid/romulus/dl3/ed/dl3_fullenclosure_moderate2tel/',
  'in_dir': '/raid/romulus/swong/mydl3/crabtest/',
  'out_dir': '/raid/romulus/swong/mydl3/crabtestbkgs/',
  'results_dir': '/raid/romulus/swong/mydl3/crabtest/results/',
  'from_run': True,
  'from_runlist': False},
 'binning': {'e_min': 0.1,
  'e_max': 100,
  'e_bins': 10,
  'off_min': 0,
  'off_max': 2.5,
  'off_bins': 20},
 'config': {'njobs': 16},
 'exclusion': {'star_cat': './Hipparcos_MAG8_1997.dat',
  'exclude_source_name': None,
  'exclude_regions': None},
 'background_selection': {'nsb_diff': 1.5,
  'el_diff': 1,
  'az_diff': 45,
  'n_tels': 4,
  'time_max': 180,
  'same_szn': True,
  'smooth': True,
  'smooth_sigma': 1,
  'time_request': 50,
  'KL_DIV': True,
  'store_KL': False},
 'run_selection': {'source_name': 'Crab',
  'pos_from_DL3': True,
  'source_ra': 83.633114,
  'source_dec': 22.014487,

# 2. Background generation

The following lines generate bkg headers for runs matching the conditions defined in `config['background_selction']`. Run-wise background generation is currently not supported but this can be found in `examples/prepare_dataset`. 

**If you wish to skip bkg generation, make sure your DL3 files with attached bkg headers are located in `config['io']['out_dir']` and skip to step 3**

## Load in our data runs

Our DL3 files are contained in the config file in `config['io']['in_dir']`- let's load those in and look at our runlist

Alternatively, if you have preprocessed anasum files on disk or only want to use selected runs in the directory, you can set `config['io']['in_dir']` to your datastore and set `config['io']['use_runlist'] = True` as well as providing a runlist (same as ED runlist; one run number per line) in `config['io']['runlist']`, which tells gammapy which runs to use.

In [6]:
data_store = DataStore.from_dir(config["io"]["in_dir"])

if config['io']['from_runlist']:
    obs_ids = np.genfromtxt(config['io']['runlist'],dtype=int,unpack=True)
    
else:
    obs_ids = data_store.obs_ids
    
print(obs_ids)

[64080 64081 64082 64083]


## Find and attach bkgs to the data runs

gammapy uses bkg files (called backgrounds) in the same way that VERITAS analyses use radial acceptances (i.e., bkg $\equiv$ radial acceptance). These normalize the counts in the ring background according to changes in camera acceptance dropping off with radial distance. 

We want to match these bkg files as closely as possible in observing conditions and time to our actual data runs, using parameters defined in our config file.

Radial acceptance is also energy dependent, so we want to define some energy bins at which to generate bkg files. We also want to bin in the X-Y camera plane.

By default, bkgs are also smoothed by a Gaussian kernel with a 1-pixel width in the camera plane

The `get_background_for_run` function takes as input a tuple with (obs_id,config) and takes the input run matching the observation ID from `config['io']['in_dir']` and outputs a DL3 file with an appended bkg to `config['io']['out_dir']`.

This prints out the number of background run hours that are found for each data run. If this is < 10, consider loosening the observing parameters in `config['background_selection']`

In [7]:
from astropy.time import Time
epochs = {
        "V4": {"tstart": Time("2000-01-01"), "tstop": Time("2009-09-13")},
        "V5": {"tstart": Time("2009-09-14"), "tstop": Time("2012-07-31")},
        "V6_2012_2013a": {"tstart": Time("2012-08-01"), "tstop": Time("2013-03-15")},
        "V6_2012_2013b": {"tstart": Time("2013-03-16"), "tstop": Time("2013-11-16")},
        "V6_2013_2014a": {"tstart": Time("2013-11-17"), "tstop": Time("2014-05-12")},
        "V6_2013_2014b": {"tstart": Time("2014-05-13"), "tstop": Time("2014-11-07")},
        "V6_2014_2015": {"tstart": Time("2014-11-08"), "tstop": Time("2015-07-31")},
        "V6_2015_2016": {"tstart": Time("2015-08-01"), "tstop": Time("2016-07-31")},
        "V6_2016_2017": {"tstart": Time("2016-08-01"), "tstop": Time("2017-07-31")},
        "V6_2017_2018": {"tstart": Time("2017-08-01"), "tstop": Time("2018-07-31")},
        "V6_2018_2019": {"tstart": Time("2018-08-01"), "tstop": Time("2019-11-12")},
        "V6_2019_2020w": {"tstart": Time("2019-11-13"), "tstop": Time("2020-05-07")},
        "V6_2020_2020s": {"tstart": Time("2020-05-08"), "tstop": Time("2020-11-04")},
        "V6_2020_2021w": {"tstart": Time("2020-11-05"), "tstop": Time("2021-04-27")},
        "V6_2021_2021s": {"tstart": Time("2021-04-28"), "tstop": Time("2021-11-16")},
        "V6_2021_2022w": {"tstart": Time("2021-11-17"), "tstop": Time("2022-05-09")},
        #  "V6_2022_2022s" : {"tstart"  :Time("2022-05-10"), "tstop" : Time("2022-11-08")},
        "V6_2022_2022s": {"tstart": Time("2022-05-10"), "tstop": Time("2025-11-08")},
    }

zeniths = [(0,10),(10,20),(20,30),(30,40),(40,45),(45,50),(50,55),(55,60)]
azimuths = [(-90,90),(90,270)]

In [8]:
# !mkdir /raid/romulus/swong/mydl3/crab_irf/

In [9]:
# use lookup table to find bkg
#path = '/raid/romulus/swong/mydl3/lookup_2d_moderate_epochs/'
#path = '/raid/romulus/swong/mydl3/lookup_2d_moderate_epochs_nosmooth/'
path='/raid/romulus/swong/mydl3/lookup_2d_hard_epochs_poly/'

in_dir = config['io']['in_dir']
out_dir = config['io']['out_dir']

if config['io']['from_runlist']:
    ids = np.genfromtxt(config['io']['runlist'],unpack=True)

try:
    observations = data_store.get_observations(required_irf='all-optional',obs_id=ids)
except NameError:
    observations = data_store.get_observations(required_irf='all-optional', obs_id=obs_ids)

for obs in tqdm(observations):
    #print(obs.obs_id)
    ze = 90 - obs.get_pointing_altaz(obs.tmid).alt.value
    az = obs.get_pointing_altaz(obs.tmid).az.value
    az = 270-az
    
    for z in zeniths:
        if (ze >= z[0]) & (ze < z[1]):
            zenith = str(f'{z[0]}-{z[1]}')
    for a in azimuths:
        if (az >= a[0]) & (az < a[1]):
            azimuth = str(f'{a[0]}-{a[1]}')
    for epoch in epochs:
        if (obs.tstart.iso > epochs[epoch]['tstart']) & (obs.tstart.iso < epochs[epoch]['tstop']):
            this_epoch = epoch
            break
            
    bkg_string = path + "bkg2d_e_0.01_100TeV_azimuth_" + azimuth + "_zenith_" + zenith + "_" + this_epoch+ ".fits"#"_nosmooth.fits"
    
    hdul = fits.open(in_dir + str(obs.obs_id) + '.fits.gz')
    if "BACKGROUND" in hdul:
            bkg_indx = hdul.index_of("BACKGROUND")
            hdul.pop(bkg_indx)
            
    bkg_hdul = fits.open(bkg_string)
            
    hdul.append(bkg_hdul[1])
    hdul.writeto(out_dir+str(obs.obs_id) + '.fits.gz', overwrite=True)

    #print(bkg_string)

  0%|          | 0/4 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/raid/romulus/swong/mydl3/crabtest/64080.fits.gz'

In [35]:
print(obs.pointing)

FixedPointingInfo:

mode:        PointingMode.POINTING
coordinates: <SkyCoord (ICRS): (ra, dec) in deg
    (83.63261075, 22.51447044)>


In [36]:
obs.pointing.get_altaz()

AttributeError: 'NoneType' object has no attribute 'scale'

for obs in tqdm(obs_ids):
    #print(obs)
    if config['io']['from_run']:
        generate_background_from_run((obs,config))
    else:
        get_background_for_run((obs,config))

## Write index files for the new DL3 directory

From now on, `config['io']['out_dir']` is the data directory containing DL3 files ready for data analysis (bkgs are appended).

In [ ]:
write_index_files(config)

## Peek at data to make sure everything worked - optional

All plots should load, in particular make sure the background rate histogram is filled, since this is the only part of the DL3 file we've changed in this notebook.

This will take a while to download for long runlists, so feel free to skip this or terminate it early if everything looks fine!

In [ ]:
from gammapy.data import DataStore
import seaborn as sns
sns.set_theme(font="Serif",style='ticks',context='paper',palette='pastel')

my_data = DataStore.from_dir(config["io"]["out_dir"])
my_observations = my_data.get_observations()


for obs in my_observations:
    print(obs.obs_id)
    obs.bkg.peek()
    obs.bkg.plot_at_energy([0.12 * u.TeV,0.3*u.TeV,5*u.TeV,10*u.TeV,30*u.TeV,50*u.TeV])
    plt.show()

# 3. Analysis

This performs a 1D (energy only) reflected regions analysis to get the spectrum and light curve (which gives an integrated flux value for all observations). Sky maps and significances are determined with a 2D (energy + acceptance) ring background analysis.

## i) Spectral analysis

Spectral analysis parameters are read in from the `spectrum` section of the config file. You can also set the analysis as either `FoV` (field of view background method) or `RE` (reflected regions). 

If a spectral shape is not provided, a power law fit is used by default.

The first plot produced shows the exclusion regions used, and the second is an $E^2 \frac{dn}{de}$ spectrum.

The spectral model will be integrated between `config["spectrum"]["emin"]` and `config["spectrum"]["emax"]` to calculate an integral flux.

In [37]:
if config['spectrum']['analysis_type'] == 'FoV':
    spectral_points,spectral_model = make_spectrum_fov(config,plot=False)
elif config['spectrum']['analysis_type'] == 'RE':
    spectral_points, spectral_model = make_spectrum_RE(config, plot=False)
else:
    print('Please select a valid analysis type')

FileNotFoundError: /raid/reedbuck/lrizk/gammapy-datasets/1.3/catalogs/Hipparcos_MAG8_1997.dat not found.

## ii) RBM analysis

Calculates counts, excess, significance, etc. using RBM normalized by the background files appended to each run.

Smaller skymap binning results in much longer runtimes. A binning of 0.01 deg is recommended to match ED, but fast analyses may be done with larger binning (gammapy default is 0.02 deg).

In [39]:
counts, background, alpha, sigma, excess_map, exposure, significance_map, exclusion_mask, alpha_map = rbm_analysis(config)

KeyError: 'aeff_max_percent'

In [ ]:
counts, background, alpha, sigma


## Plot results

Makes RBM maps (significance & excess) and RBM significance distribution.

These plots are saved with the prefix defined in: `config['plot_names']`

**If you did not compute a spectrum, you can still plot the RBM plots by setting the optional parameter `spectrum=False` in the function call and setting `spectral_points = []`**

In [40]:
spectral_points=[]
rbm_plots(config, spectral_points, excess_map, significance_map, exclusion_mask, alpha_map, spectrum=False)

NameError: name 'excess_map' is not defined

# 4. Analysis output

Write the analysis output (currently formatted for inputting into the gammapy validation page: https://veritas.sao.arizona.edu/wiki/Gammapy_Validation).

**If you did not compute a spectrum, you can still plot the RBM plots by setting the optional parameter `spectrum=False` in the function call and `spectral_model=[]`**

This is saved in yaml format to the path defined in `config['results_file']`.


**You should also upload your config file after validating a source for reproducibility!**

In [ ]:
spectral_model=[]
write_validation_info(config,spectral_model,counts,background,alpha,sigma,exposure,spectrum=False)

In [ ]:
from gammapy.maps import WcsNDMap
from astropy.io import fits
from astropy.wcs import WCS

hdul = fits.open('/raid/romulus/swong/data/analysis/Results/v490/crabv4rbm/crabv4.fits')

ed_map = WcsNDMap.from_hdu(hdul[5])

gammapy_map = significance_map.reduce('energy')
sub_map = np.subtract(gammapy_map,ed_map)

In [ ]:
sns.set_theme(font="Serif",style='ticks',context='talk',palette='pastel')

w = WCS(hdul[5].header)

fig,ax = plt.subplots()
fig.set_figwidth(7)
fig.set_figheight(7)
#plt.subplot(projection=w)
plt.title('gammapy - ED')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
plt.imshow(sub_map,vmin=-5,vmax=5)

ndeg = 2
r = ndeg/0.01
circle = plt.Circle((200,200), r ,fill = False,color='k',lw=1,ls='--',label=f'r={ndeg} deg')

ax.add_patch(circle)
plt.legend(loc='upper right')

plt.colorbar(label='Significance',shrink=0.85)

In [ ]:
offset = np.arange(1,2.2,0.1)
mu = [-0.16,-0.17,-0.16,-0.17,-0.15,-0.15,-0.16,-0.16,-0.15,-0.15,-0.17,-0.18,-0.21]
sigma = [1.13,1.11,1.11,1.11,1.09,1.08,1.08,1.05,1.05,1.05,1.05,1.05,1.05]

In [ ]:
plt.plot(offset,mu,'o',ms=7,color='salmon')
plt.xlabel('offset [deg]')
plt.ylabel('mu')

In [ ]:
plt.plot(offset,sigma,'o',ms=7,color='plum')
plt.xlabel('offset [deg]')
plt.ylabel('sigma')